In [20]:
import os
import numpy as np
import pandas as pd
from skimage.util.montage import montage2d
import matplotlib.pyplot as plt

In [4]:
def load_and_format(in_path):
    out_df = pd.read_json(in_path)
    out_images = out_df.apply(lambda c_row: [np.stack([c_row['band_1'],c_row['band_2']], -1).reshape((75,75,2))],1)
    out_images = np.stack(out_images).squeeze()
    return out_df, out_images
train_df, train_images = load_and_format('../train.json')
print('training', train_df.shape, 'loaded', train_images.shape)
test_df, test_images = load_and_format('../test.json')
print('testing', test_df.shape, 'loaded', test_images.shape)
train_df.sample(3)

training (1604, 5) loaded (1604, 75, 75, 2)
testing (8424, 4) loaded (8424, 75, 75, 2)


,band_1,band_2,id,inc_angle,is_iceberg
723,"[-20.983963, -18.986147, -18.636415, -21.13527...","[-28.293722, -27.309444, -26.149689, -26.14977...",a47b7dee,32.2297,1
791,"[-22.191774, -21.276672, -24.626541, -28.40771...","[-25.428791, -26.312912, -25.713694, -24.88588...",93beaaaa,41.8704,0
111,"[-25.652521, -27.236193, -21.93988, -20.547207...","[-34.367096, -32.27243, -31.112637, -33.611458...",2b88818c,42.5591,1


In [11]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
X_train, X_test, y_train, y_test = train_test_split(train_images,
                                                   to_categorical(train_df['is_iceberg']),
                                                    random_state = 2017,
                                                    test_size = 0.5
                                                   )
print('Train', X_train.shape, y_train.shape)
print('Validation', X_test.shape, y_test.shape)

Using TensorFlow backend.


Train (802, 75, 75, 2) (802, 2)
Validation (802, 75, 75, 2) (802, 2)


In [16]:
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, Dropout, MaxPooling2D, GlobalMaxPooling2D, Dense
simple_cnn = Sequential()
simple_cnn.add(BatchNormalization(input_shape = (75, 75, 2)))
for i in range(4):
    simple_cnn.add(Conv2D(8*2**i, kernel_size = (3,3)))
    simple_cnn.add(MaxPooling2D((2,2)))
simple_cnn.add(GlobalMaxPooling2D())
simple_cnn.add(Dropout(0.5))
simple_cnn.add(Dense(8))
simple_cnn.add(Dense(2, activation = 'softmax'))
simple_cnn.compile(optimizer='sgd', loss = 'binary_crossentropy', metrics = ['accuracy'])
simple_cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 75, 75, 2)         8         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 73, 73, 8)         152       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 8)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 16)        1168      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 32)        4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 7, 7, 32)          0         
__________

In [17]:
simple_cnn.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 100, shuffle = True)

Train on 802 samples, validate on 802 samples
Epoch 1/100
802/802 [==============================] - 0s - loss: 0.7157 - acc: 0.5673 - val_loss: 0.9253 - val_acc: 0.4788
Epoch 2/100
802/802 [==============================] - 0s - loss: 0.6719 - acc: 0.5898 - val_loss: 1.5169 - val_acc: 0.4788
Epoch 3/100
802/802 [==============================] - 0s - loss: 0.6225 - acc: 0.6322 - val_loss: 1.2257 - val_acc: 0.5212
Epoch 4/100
802/802 [==============================] - 0s - loss: 0.6123 - acc: 0.6322 - val_loss: 0.9878 - val_acc: 0.5698
Epoch 5/100
802/802 [==============================] - 0s - loss: 0.5961 - acc: 0.6334 - val_loss: 0.6972 - val_acc: 0.6746
Epoch 6/100
802/802 [==============================] - 0s - loss: 0.5760 - acc: 0.6671 - val_loss: 0.7762 - val_acc: 0.6035
Epoch 7/100
802/802 [==============================] - 0s - loss: 0.5587 - acc: 0.6820 - val_loss: 0.6380 - val_acc: 0.6496
Epoch 8/100
802/802 [==============================] - 0s - loss: 0.5257 - acc: 0.7195

In [18]:
test_predictions = simple_cnn.predict(test_images)

In [19]:
pred_df = test_df[['id']].copy()
pred_df['is_iceberg'] = test_predictions[:,1]
pred_df.to_csv('predictions.csv', index = False)
pred_df.sample(3)

,id,is_iceberg
4936,f084666d,0.070851
7540,16105a8c,0.280635
4504,719ecd9f,0.003676
